# 기존 모델 추론 속도 측정
- 이 로직 실행하려면 numpy 라이브러리 때문에 세션 새로 시작한 후 실행해야함

In [1]:
# google drive mount
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import torch
import time

In [3]:
## for from kobert_transformers.utils import get_tokenizer
!pip install kobert-transformers

In [4]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
!pip install mxnet # 파이썬에서 실행 안 됨
!pip install transformers
!pip install torch

In [5]:
## for from kobert.pytorch_kobert import get_pytorch_kobert_model
#!pip install onnxruntime==1.8.0
!git clone https://github.com/SKTBrain/KoBERT.git
%cd KoBERT
!pip install boto3

fatal: destination path 'KoBERT' already exists and is not an empty directory.
/content/KoBERT


In [6]:
## for import gluonnlp as nlp
!pip install gluonnlp

In [11]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.19 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.20 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.31.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy

In [7]:
# import modules

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

import numpy as np

from tqdm import tqdm, tqdm_notebook
from kobert_transformers.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup


In [8]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-haef50na/kobert-tokenizer_24c11782001845e2bc8ed409096ac553
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-haef50na/kobert-tokenizer_24c11782001845e2bc8ed409096ac553
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-iw_ss7ml
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-iw_ss7ml
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.15.18-py2.py3-none-any.whl.metadata (5.3 kB)
  Using cached mxnet-1.7.0.post2-py2.py3-none-manylinux2014_x86_64.whl.metadata (3.0 kB)
INFO: pip is looking 

In [9]:
# for 데이터프레임 내 non-string을 string으로 변환
import pandas as pd
from transformers import BertModel, BertTokenizer
from kobert_tokenizer import KoBERTTokenizer  # 1-3-2. BERTDataset 형식 변환 바로 밑 설치 코드 실행 후 실행하면 잘 돌아감

In [10]:
# BERT 모델과 Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()
MAX_LEN = 64

/content/KoBERT/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [11]:
class BERTClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=2, dr_rate=None, params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        # TorchScript 호환 방식으로 변경
        batch_size, seq_len = token_ids.size()
        attention_mask = torch.arange(seq_len, device=token_ids.device).expand(batch_size, seq_len)
        attention_mask = (attention_mask < valid_length.unsqueeze(1)).float()
        return attention_mask

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        _, pooler = self.bert(
            input_ids=token_ids,
            token_type_ids=segment_ids.long(),
            attention_mask=attention_mask.float().to(token_ids.device),
            return_dict=False,
        )
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [12]:
# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# 학습된 모델 로드
model_path = "/content/gdrive/MyDrive/crimefin/model/model.pt"
state_dict = torch.load(model_path, map_location=device)
bert_state_dict = state_dict['bert']
trained_model = BERTClassifier(bertmodel, num_classes=2, dr_rate=0.5).to(device)
trained_model.load_state_dict(bert_state_dict, strict=False)
trained_model.eval()


<ipython-input-13-caf69283d44d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_location=device)


BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [16]:
# # 입력 데이터 생성
# batch_size = 1
# example_token_ids = torch.randint(0, 30522, (batch_size, MAX_LEN), dtype=torch.long).to(device)
# example_valid_length = torch.tensor([MAX_LEN] * batch_size, dtype=torch.long).to(device)
# example_segment_ids = torch.zeros((batch_size, MAX_LEN), dtype=torch.long).to(device)
# 입력 데이터 생성 (샘플)
batch_size = 1
max_len = 64

# Get vocabulary size from the loaded model
vocab_size = trained_model.bert.embeddings.word_embeddings.weight.shape[0]
# word_embeddings is assumed to be the embedding layer within your model

# Ensure token IDs are within vocabulary range
example_token_ids = torch.randint(0, vocab_size, (batch_size, max_len), dtype=torch.long).to(device)
example_valid_length = torch.tensor([max_len] * batch_size, dtype=torch.long).to(device)
example_segment_ids = torch.zeros((batch_size, max_len), dtype=torch.long).to(device)

In [17]:
# 추론 속도 측정
start_time = time.time()
with torch.no_grad():
    for _ in range(100):  # 100번 반복 실행
        trained_model(example_token_ids, example_valid_length, example_segment_ids)
end_time = time.time()

In [18]:
inference_time = (end_time - start_time) / 100
print(f"Ko-BERT 모델 평균 추론 시간: {inference_time:.6f} 초")

Ko-BERT 모델 평균 추론 시간: 0.274553 초


# TorchScript 모델 추론 속도 측정

In [1]:
import torch
import time

In [3]:
# TorchScript 모델 로드
torchscript_model_path = "/content/gdrive/MyDrive/crimefin/model/torchscript_kobert.pt"
scripted_model = torch.jit.load(torchscript_model_path, map_location="cpu")
scripted_model.eval()

RecursiveScriptModule(
  original_name=BERTClassifier
  (bert): RecursiveScriptModule(
    original_name=BertModel
    (embeddings): RecursiveScriptModule(
      original_name=BertEmbeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (encoder): RecursiveScriptModule(
      original_name=BertEncoder
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=BertLayer
          (attention): RecursiveScriptModule(
            original_name=BertAttention
            (self): RecursiveScriptModule(
              original_name=BertSdpaSelfAttention
              (query): RecursiveScriptModule(original_name=

In [6]:
# # 입력 데이터 생성 (샘플)
# batch_size = 1
# max_len = 64
# example_token_ids = torch.randint(0, 30522, (batch_size, max_len), dtype=torch.long)
# example_valid_length = torch.tensor([max_len] * batch_size, dtype=torch.long)
# example_segment_ids = torch.zeros((batch_size, max_len), dtype=torch.long)
# 입력 데이터 생성 (샘플)
batch_size = 1
max_len = 64

# Get vocabulary size from the loaded model
vocab_size = scripted_model.bert.embeddings.word_embeddings.weight.shape[0]
# word_embeddings is assumed to be the embedding layer within your model

# Ensure token IDs are within vocabulary range
example_token_ids = torch.randint(0, vocab_size, (batch_size, max_len), dtype=torch.long)
example_valid_length = torch.tensor([max_len] * batch_size, dtype=torch.long)
example_segment_ids = torch.zeros((batch_size, max_len), dtype=torch.long)

In [7]:
# 추론 속도 측정
start_time = time.time()
with torch.no_grad():
    for _ in range(100):  # 100번 반복 실행
        scripted_model(example_token_ids, example_valid_length, example_segment_ids)
end_time = time.time()

In [8]:
torchscript_inference_time = (end_time - start_time) / 100
print(f"TorchScript 모델 평균 추론 시간: {torchscript_inference_time:.6f} 초")

TorchScript 모델 평균 추론 시간: 0.261753 초


# ONNX 모델 추론 속도 측정

In [10]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [11]:
import onnxruntime
import numpy as np
import time

In [12]:
# ONNX 모델 로드
onnx_model_path = "/content/gdrive/MyDrive/crimefin/model/onnx_kobert.onnx"
onnx_session = onnxruntime.InferenceSession(onnx_model_path)

In [15]:
# # ONNX 입력 이름 확인
# input_names = {inp.name: inp.name for inp in onnx_session.get_inputs()}
# input_feed = {
#     input_names["input_ids"]: example_token_ids,
#     input_names["valid_length"]: example_valid_length,
#     input_names["segment_ids"]: example_segment_ids,
# }
# ONNX 입력 이름 확인
input_names = {inp.name: inp.name for inp in onnx_session.get_inputs()}
input_feed = {
    input_names["input_ids"]: example_token_ids.cpu().numpy(), # Convert to NumPy array
    input_names["valid_length"]: example_valid_length.cpu().numpy(), # Convert to NumPy array
    input_names["segment_ids"]: example_segment_ids.cpu().numpy(), # Convert to NumPy array
}

In [16]:
# 추론 속도 측정
start_time = time.time()
for _ in range(100):  # 100번 반복 실행
    onnx_session.run(None, input_feed)
end_time = time.time()

In [17]:
onnx_inference_time = (end_time - start_time) / 100
print(f"ONNX 모델 평균 추론 시간: {onnx_inference_time:.6f} 초")

ONNX 모델 평균 추론 시간: 0.200988 초


# 성능 비교

In [18]:
print(f"TorchScript 모델 평균 추론 시간: {torchscript_inference_time:.6f} 초")
print(f"ONNX 모델 평균 추론 시간: {onnx_inference_time:.6f} 초")

if torchscript_inference_time < onnx_inference_time:
    print("TorchScript 모델이 더 빠릅니다.")
else:
    print("ONNX 모델이 더 빠릅니다.")


TorchScript 모델 평균 추론 시간: 0.261753 초
ONNX 모델 평균 추론 시간: 0.200988 초
ONNX 모델이 더 빠릅니다.
